In [30]:
import json
import tweepy
import pandas as pd
from tweepy import OAuthHandler
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import time

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [4]:
con_key = 'uz4wSx2vt2PlKbWCOlIpKbBZo'
con_secret = 'lq61FxYRvJPwLjjRlUf0rSTsdXf86iijHYeGfrUdi7EhZzWsWv'
acc_token = '1038510225786122247-zLnTCjChQ0rLUHevueL0CN4ygdY1Me'
acc_token_secret = 'W7lqUevzXRpzI4zQA6niMDR85r9SCo3sCQGaPA4wg6DS6'

In [5]:
auth = OAuthHandler(con_key, con_secret)
auth.set_access_token(acc_token, acc_token_secret)
new = tweepy.API(auth)

results = new.search(q="@FLGovScott" , geocode = , count = 1, tweet_mode = 'extended')

In [18]:
results = new.search(q="@FLGovScott" ,  , count = 1 , geocode = '27.967042,-81.471955,320km' , tweet_mode = 'extended')

In [21]:
for result in results:
    print(result.created_at)
    print('\n')
    print('1::: fullText:: ' , result.full_text)
    print('2::: geo:: ' , result.geo)
    print('3::: place:: ' , result.place)
    print('4::: coordinates:: ' , result.coordinates)
    print('5::: retweet Count::  ' , result.retweet_count)
    print('6::: retweeted:: ' , result.retweeted)
    print('7::: lang:: ' , result.lang)
    print('8::: ID:: ' , result.id_str)
    print(result.contributors)
    print('9::: TextRange:: ' , result.entities)
    print('10::: User Location:: ', result.user.location)
    print('9::: TextRange:: ' ,result.metadata)
    print('1000::: USER:: ' , result.user)
    result.l
   # result.full_text

2018-09-30 17:29:35


1::: fullText::  RT @FLGovScott: During their past 88 years, @Publix has expanded to 1100 stores &amp; added countless jobs, including 700 new jobs with their r…
2::: geo::  None
3::: place::  None
4::: coordinates::  None
5::: retweet Count::   109
6::: retweeted::  False
7::: lang::  en
8::: ID::  1046452007614205953
None
9::: TextRange::  {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'FLGovScott', 'name': 'Rick Scott', 'id': 131546062, 'id_str': '131546062', 'indices': [3, 14]}, {'screen_name': 'Publix', 'name': 'Publix', 'id': 14784550, 'id_str': '14784550', 'indices': [44, 51]}], 'urls': []}
10::: User Location::  
9::: TextRange::  {'iso_language_code': 'en', 'result_type': 'recent'}
1000::: USER::  User(_api=<tweepy.api.API object at 0x1a1129fda0>, _json={'id': 61171346, 'id_str': '61171346', 'name': 'Debbie McNally', 'screen_name': 'shrtcake60', 'location': '', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, '

In [19]:
results

[Status(_api=<tweepy.api.API object at 0x1a1129fda0>, _json={'created_at': 'Sun Sep 30 17:29:35 +0000 2018', 'id': 1046452007614205953, 'id_str': '1046452007614205953', 'full_text': 'RT @FLGovScott: During their past 88 years, @Publix has expanded to 1100 stores &amp; added countless jobs, including 700 new jobs with their r…', 'truncated': False, 'display_text_range': [0, 144], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'FLGovScott', 'name': 'Rick Scott', 'id': 131546062, 'id_str': '131546062', 'indices': [3, 14]}, {'screen_name': 'Publix', 'name': 'Publix', 'id': 14784550, 'id_str': '14784550', 'indices': [44, 51]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'u

## This is the Begining

In [42]:
def Senter(tweets , Cadidate , DF):
    for tweet in tweets:
        sentiment = '0'
        #CHECK IF IT IS AN ELECTION TWEET
        #SO WE NEED TO FIND A WAY TO DO THIS!!!!
        if tweet.lang == 'en':
            #send to english sentiment analyser
            sentiment = analyzer.polarity_scores(tweet.full_text)
        else:
            #send to this sentiment analyser
            sentiment = analyzer.polarity_scores(tweet.full_text)
        DF.append({'Candidate': '%s'.format(Cadidate) , 'Time' : result.created_at ,
               'Text': result.full_text , 'userLocation': result.user.location , 
               'coordinates': result.coordinates , 'retweeted': result.retweeted , 
              'Language' : result.lang , 'Sentiment': sentiment} , ignore_index = True)

In [ ]:
columns = ['Candidate' , 'Time' , 'Text' , 'userLocation' , 'coordinates' , 'retweeted' , 'language' , 'Sentiment' ]
AZ_path_name = 'iCloud Drive/Desktop/Politter/AZOutput'
FL_path_name = 'iCloud Drive/Desktop/Politter/FLOutput'
consume_count = 1
FL = pd.DataFrame(columns = columns)
AZ = pd.DataFrame(columns = columns)

while True:
    
    if time.localtime().tm_hour == 24:
        FL.to_csv(FL_path_name , sep = ',')
        AZ.to_csv(AZ_path_name , sep = ',')
      
    
    #Florida Race
    Rick = new.search(q="@FLGovScott" , count = consume_count  , geocode = '27.967042,-81.471955,320km' , tweet_mode = 'extended')    
    if len(Rick) == 0:
        time.sleep(60 * 15)
        Rick = new.search(q="@FLGovScott" , count = consume_count  , geocode = '27.967042,-81.471955,320km' , tweet_mode = 'extended')    
    Senter(Rick , 'GOP:RICK SCOTT' , FL)
    
    Bill = new.search(q="@SenBillNelson" , count = consume_count  , geocode = '27.967042,-81.471955,320km' , tweet_mode = 'extended')    
    if len(Bill) == 0:
        time.sleep(60 * 15)
        Bill = new.search(q="@SenBillNelson" , count = consume_count  , geocode = '27.967042,-81.471955,320km' , tweet_mode = 'extended')    
    Senter(Bill , 'DEM:BILL NELSON' , FL)
    
    
    #Arazona Race
    Martha = new.search(q="@RepMcSally" , count = consume_count  , geocode = '34.383026,-111.694738,400km' , tweet_mode = 'extended')    
    if len(Martha) == 0:
        time.sleep(60 * 15)
        Martha = new.search(q="@RepMcSally" , count = consume_count  , geocode = '34.383026,-111.694738,400km' , tweet_mode = 'extended')    
    Senter(Martha , 'GOP:MARTHA MCSALLY' , AZ)
    
    Kyrsten =  new.search(q="@RepSinema" , count = consume_count  , geocode = '34.383026,-111.694738,400km' , tweet_mode = 'extended')    
    if len(Kyrsten) == 0:
        time.sleep(60 * 15)
    Kyrsten = new.search(q="@RepSinema" , count = consume_count  , geocode = '34.383026,-111.694738,400km' , tweet_mode = 'extended')    
    Senter(Kyrsten , 'DEM:KYRSTEN SINEMA' , AZ)